### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [18]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [19]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [27]:
# Number of movies
print("No of movies",movies.movie.nunique())

# Number of unique genre
gen=[]
for i in movies.genre:
    try:
        gen.extend(i.split("|"))
    except AttributeError:
        pass
gen=set(gen)
print("No of unique genre",len(gen))

No of movies 33959
No of unique genre 28


In [28]:
print("Ratings mean",reviews.rating.mean())
print("Ratings max",reviews.rating.max())
print("Ratings min",reviews.rating.min())
print("no. of ratings null value",reviews.rating.isna().sum(axis = 0))


Ratings mean 7.31587021456
Ratings max 10
Ratings min 0
no. of ratings null value 0


In [22]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 
'The number of ratings in the dataset':
'The number of different genres':
'The number of unique users in the dataset':
'The number missing ratings in the reviews dataset':
'The average rating given across all ratings':
'The minimum rating given across all ratings':
'The maximum rating given across all ratings':
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


SyntaxError: invalid syntax (<ipython-input-22-0cfdee39ed6f>, line 6)

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [23]:
# and another
movies['year']=movies['movie'].apply(lambda x:x.split('(')[1][:-1]).astype('int32')
movies['movie']=movies['movie'].apply(lambda x:x.split('(')[0])
bins = [1800,1900,2000,2050]
labels = ['1800s','1900s','2000s']
movies['binned'] = pd.cut(movies['year'],bins=bins, labels=labels)
df=pd.get_dummies(movies['binned'])
movies=pd.concat([movies,df],axis=1)
movies.head()

,movie_id,movie,genre,year,binned,1800s,1900s,2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,1800s,1,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1895,1800s,1,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1896,1800s,1,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1895,1800s,1,0,0
4,0000091,Le manoir du diable,Short|Horror,1896,1800s,1,0,0


In [47]:
import re
def search_gen(val):
    try:
        if val.find(i)>-1:
            return 1
        else:
            return 0
            
    except AttributeError:
        return 0

In [48]:
for i in gen:
    movies[i]=movies.genre.apply(search_gen)

In [49]:
movies.head()

,movie_id,movie,genre,year,binned,1800s,1900s,2000s,Adult,Action,...,Documentary,Biography,Adventure,Sci-Fi,Short,Thriller,Animation,Music,Fantasy,Horror
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,1800s,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1895,1800s,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1896,1800s,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1895,1800s,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1896,1800s,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [50]:
movies.iloc[0]

movie_id                                       0000008
movie          Edison Kinetoscopic Record of a Sneeze 
genre                                Documentary|Short
year                                              1894
binned                                           1800s
1800s                                                1
1900s                                                0
2000s                                                0
Adult                                                0
Action                                               0
Drama                                                0
Crime                                                0
Reality-TV                                           0
War                                                  0
Talk-Show                                            0
Sport                                                0
News                                                 0
Western                                              0
Family    

In [53]:
movies.drop('binned',axis=1,inplace=True)
movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s,Adult,Action,Drama,...,Documentary,Biography,Adventure,Sci-Fi,Short,Thriller,Animation,Music,Fantasy,Horror
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1895,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1896,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()